In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install category_encoders

In [ ]:
!pip install mutagen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import glob
import mutagen
from pydub import AudioSegment
from matplotlib import pyplot as plt
from mutagen.wave import WAVE
import sys
import librosa
import os

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/dsl_data.zip', 'r') 
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
import pandas as pd
dev= pd.read_csv('/tmp/development.csv')
dev.shape

(9854, 10)

In [ ]:
import pandas as pd
val= pd.read_csv('/tmp/evaluation.csv')
val.shape

(1455, 8)

In [ ]:
df= pd.concat([dev, val], axis= 0)
df.shape

(11309, 10)

In [ ]:
df['audio_name'] = df['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
df.drop('path', inplace=True, axis=1)
df.shape

(11309, 10)

In [ ]:
#####encode numerical columns
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
columns_to_encode = ['gender', 'First Language spoken', 'Current language used for work/school']
encoded_df = pd.DataFrame(encoder.fit_transform(df[columns_to_encode]).toarray(), columns= encoder.get_feature_names_out(columns_to_encode))

In [ ]:
df1= df.join(encoded_df)
df2= df1.drop(['gender', 'First Language spoken', 'Current language used for work/school'], axis= 1)

In [ ]:
#####encode ordinal columns
from category_encoders import OrdinalEncoder 
maplist = [{'col': 'Self-reported fluency level ', 'mapping': {'native': 3, 'advanced': 2, 'intermediate': 1, 'basic': 0}},
           {'col': 'ageRange', 'mapping': {'65+': 2, '41-65': 1, '22-40': 0}}]
oe = OrdinalEncoder(mapping= maplist)
df3= oe.fit_transform(df2)

In [ ]:
df3.to_csv('/content/drive/MyDrive/dataset_dsl_encode.csv')

Features

In [ ]:
import glob
pathtrain =[]
pathtest =[]
path_nonzero = []
for file in glob.glob('/tmp/audio/speakers/*'):
  for i in glob.glob(file + '/*.wav'):
    id= i.split('/')[-1].split('.')[0]
    if (list(df3['action'][df3['audio_name']== id].isna())[0]): 
      pathtest.append(i)
    else:
      pathtrain.append(i)

print(len(pathtrain),len(pathtest))


9854 1455


In [ ]:
def sta(data):
  dat_min = np.min(data, axis= 1) 
  dat_max = np.max(data, axis= 1)
  dat_std = np.std(data, axis= 1)
  dat_median = np.median(data ,axis= 1)
  dat_mean = np.mean(data, axis= 1)
  dat_qrl1 = np.percentile(data, 25 , axis= 1) 
  dat_qrl3 = np.percentile(data, 75 , axis= 1)
  dat_lower_q = np.quantile(data, 0.25, interpolation="lower" , axis= 1) 
  dat_higher_q = np.quantile(data, 0.75, interpolation="higher" , axis= 1) 
  return np.concatenate([dat_min,dat_max, dat_mean, dat_std, dat_median, dat_qrl1,dat_qrl3 ,dat_lower_q,dat_higher_q ])

In [ ]:
def feature_extract(path):
    y, sr = librosa.load(path, sr=16000 , offset=0.0, duration=5.0)
    ###padding to 5 seconds
    current_length = len(y)
    padding = int(5 * sr) - current_length
    y_pad = np.pad(y, (0, padding), mode='constant')
    ##### extracting featuers
    mfcc = librosa.feature.mfcc(y= y_pad, sr=sr, n_fft=256, hop_length=128, n_mfcc= 80, fmax= 8000)
    feature1= sta(mfcc)
    all= np.concatenate([feature1])   
    return all

In [ ]:
lis= []
dic= {}
for i in pathtrain:
  id= i.split('/')[-1].split('.')[0]
  feature= feature_extract(i)
  lis.append(feature)
  dic[id]= feature

In [ ]:
data= pd.DataFrame(dic.values(), index= dic.keys())
data['audio_name']= dic.keys()
data.shape

(9854, 721)

In [ ]:
train= pd.merge(df3, data, on= ['audio_name'])
train.shape

(9854, 738)

In [ ]:
train= train.drop(['audio_name', 'speakerId'], axis= 1)
train.shape

(9854, 736)

In [ ]:
def merge(a, b):
  c= str(a) + str(b)
  return c
train['label']= train.apply(lambda row : merge(row['action'], row['object']), axis = 1)
train= train.drop(['action', 'object'], axis= 1)
train.shape

(9854, 735)

In [ ]:
######target encoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
tar_label= pd.DataFrame(le.fit_transform(train['label']), columns= {'tar_label'})

train = train.join(tar_label)
train= train.drop(['label'], axis= 1)
train.shape

(9854, 735)

In [ ]:
X= train.drop(['tar_label'], axis= 1)
y= train['tar_label']

In [ ]:
print(X.shape, y.shape)

(9854, 734) (9854,)


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(X, y, test_size= 0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import metrics

In [ ]:
####grid search on XGB 
par = {'max_depth':[4, 5, 6],
       'min_child_weight' : [2, 4, 6],
       'min_samples_split':[2, 3, 4],
       'sampling_method': ['uniform', 'gradient_based']}
xgb= xgb.XGBClassifier(class_weight='balanced')
xgb_cv= GridSearchCV(xgb, param_grid= par, cv=StratifiedKFold(n_splits=5))
xgb_cv.fit(X,y)

print("Best parameters found: ", xgb_cv.best_params_)

In [ ]:
from IPython.core import macro
xgb= xgb.XGBClassifier(class_weight='balanced', max_depth= 5, min_child_weight= 4, min_samples_split= 3, sampling_method= 'uniform')
xgb.fit(x_train, y_train)
p=xgb.predict(x_test)
print(metrics.f1_score(p ,y_test , average= 'macro'), metrics.accuracy_score(p , y_test ))

0.4918100606570558 0.5038051750380518


Test

In [ ]:
#####extracting features of evaluation set
lis= []
dic= {}
for i in pathtest:
  id= i.split('/')[-1].split('.')[0]
  feature= feature_extract(i)
  lis.append(feature)
  dic[id]= feature

In [ ]:
data= pd.DataFrame(dic.values(), index= dic.keys())
data['audio_name']= dic.keys()

In [ ]:
test= pd.merge(df3, data, on= ['audio_name'])
test.shape

(1455, 738)

In [ ]:
test= test.drop(['audio_name', 'speakerId', 'action', 'object'], axis= 1)
test.shape

(1455, 734)

In [ ]:
pred= xgb.predict(test)

In [ ]:
predict= pd.DataFrame(le.inverse_transform(list(pred)), columns= {'Predicted'})

In [ ]:
predict.to_csv('/content/drive/MyDrive/XGBoost.csv')